<a href="https://colab.research.google.com/github/Aloukik21/Dataset/blob/master/Assignment_3_using_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import tensorflow.compat.v1 as tf
from keras.models import Model 
import keras
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.callbacks import Callback, LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import print_summary, to_categorical
from keras import backend as K
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
import sys
import os
import cv2 
from PIL import Image
import numpy as np
from skimage.transform import resize
from skimage import data, io, filters
from skimage.transform import rescale
import time
from sklearn.model_selection import StratifiedShuffleSplit

In [0]:
NUM_CLASSES = 100
num_classes = 100

In [0]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [0]:


x_train = np.array([cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC) for img in x_train])
x_test = np.array([cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC) for img in x_test])


In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(50000, 224, 224, 3)
(10000, 224, 224, 3)
(50000, 1)


In [0]:
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train = x_train / 255
x_test = x_test / 255


In [14]:
#x_train = x_train[1:200]
#label_train = label_train[1:200]
#x_test = x_test[1:300]
#label_test = label_test[1:300]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

#x = []
#for i in range(len(x_train)):
    
  # img = cv2.resize(x_train[i], (224, 224))
  # x.append(img)

(50000, 224, 224, 3)
(50000, 100)
(10000, 224, 224, 3)
(10000, 100)


In [15]:
print(x.shape)

NameError: ignored

In [0]:
def Alexnet(n_class=100, init_lr = 0.01):
    
    # layer 1
    inputs = Input((224,224,3))
    conv1 = Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),activation = 'relu',strides=(4,4), padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    
    pool1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv1)
    
    # layer 2
    conv2 = Conv2D(filters=256,kernel_size=(5,5),activation ='relu',strides=(1,1), padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv2)
    
    # layer 3
    conv3 = Conv2D(filters=384,kernel_size=(3,3),activation = 'relu',strides=(4,4), padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv3)
    
    # layer 4
    conv4 = Conv2D(filters=384,kernel_size=(3,3),activation = 'relu',strides=(4,4), padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    
    # layer 5
    conv5 = Conv2D(filters=256,kernel_size=(3,3),activation = 'relu',strides=(4,4), padding='same')(conv4)
    conv5 = Dropout(0.2)(conv5)
    
    pool5 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv5)
    # flattern layer and fcc layer
    conv6 = Flatten() (pool5)
    
    conv6 = Dense(4096) (conv6)
    conv6 = Activation('relu') (conv6)
    conv6 = Dropout(0.5)(conv6)
  
    
    
    # last o/p 
    conv8 = Dense(n_class)(conv6)
    conv8 = Activation('softmax')(conv8)
    
    model = Model(input=inputs, output=conv8)
    
    
    model.compile(loss="categorical_crossentropy", optimizer= Adam(lr=init_lr) , metrics=['accuracy'] )
    
    
    return model

In [18]:
model = Alexnet()
model.summary()
#model.load_weights('alexnet_weights_1.h5', by_name=True)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 96)        34944     
_________________________________________________________________
dropout_1 (Dropout)          (None, 56, 56, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 256)       614656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 28, 28, 256)       0         
___

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


In [0]:
checkpoint = ModelCheckpoint('best_model_simple.h5',  #--- only the best model will be forwared in next epoch
                             monitor='val_loss', 
                             verbose=0,
                             save_best_only= True, 
                             mode='auto') 

In [0]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=1.0e-4), 
              metrics = ['accuracy'])

In [0]:
model_details = model.fit(x_train, y_train,
                    batch_size = 128, # number of samples 
                    epochs = 100, # number of epochs
                    validation_data= (x_test, y_test),
                    callbacks=[checkpoint],
                    verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 50000 samples, validate on 10000 samples
Epoch 1/100





50000/50000 [==============================] - 50s 1ms/step - loss: 4.3006 - acc: 0.0423 - val_loss: 4.0432 - val_acc: 0.0900
Epoch 2/100
42112/50000 [========================>.....] - ETA: 5s - loss: 3.9092 - acc: 0.0890

In [0]:
model.save_weights('my_model_weights.h5')

In [0]:
#tes1
#20 epocs
#accu =0.3336
#val_accu=0.2794

#test2 removed dropout from cnn
#20 epocs
#accu =0.3143
#val_accu=0.2064

#test3 --- changes values of kernel and strides
#20 epocs
#accu =0.4832 
#val_accu=0.2790 #------------overfitting